# Laboratorijska vaja: Zgodovinske šifre

Namen laboratorijske vaje se je spoznati z izbranimi zgodovinskimi šiframi, 
jih implementirati, nato pa jih tudi uspešno napasti. 

Podrobnosti posameznih šifer poiščite v prosojnicah s predavanj.

## Pomožne funkcije in spremenljivke

Za začetek v globalno spremenljivko `ALPHABET` zapišimo abecedo znakov. 
V našem primeru abeceda vsebuje male slovenske črke in presledek: primere bomo
poenostavili, tako da bomo izpuščali velike črke, števke, ločila in ostale znake.

Funkcija `normalize` filtrira poljuben niz in iz njega odstrani vse znake, ki niso
del naše abecede. Funkcija `valid_input` vrne `True`, če in samo če so vsi znaki v
podanem niz del naše abecede.

In [ ]:
ALPHABET = 'abcčdefghijklmnoprsštuvzž '


def normalize(msg):
    msg = " ".join(msg.lower().split())
    return "".join(x for x in msg if x in ALPHABET)


def valid_input(data):
    return all(x in ALPHABET for x in data)

## Naloga 1: Zamična šifra

Pri zamični šifri znake besedila zamaknemo za toliko mest kot določa črka v ključu. Pri implementaciji si pomagajte z globalno spremenljivko `ALPHABET`.

Metoda `enc_shift(key, pt)` naj implementira šifrirni algoritem, metoda `dec_shift(key, ct)` pa dešifrirnega.

In [ ]:
def enc_shift(key, pt):
    pass

In [ ]:
assert enc_shift("a", "dober dan") == "dober dan"
assert enc_shift("b", "dober dan") == "epcfsaebo"
assert enc_shift("c", "dober dan") == "frčgšbfcp"
assert enc_shift("č", "dober dan") == "gsdhtcgčr"

In [ ]:
def dec_shift(key, ct):
    pass

In [ ]:
for key in ALPHABET:
    assert dec_shift(key, enc_shift(key, "pozdravljen svet")) == "pozdravljen svet"

## Naloga 2: Implementacija Vigenerjeve šifre

Implementirajte šifrirni in dešifrirni algoritem za Vigenerjevo šifro. Podrobnosti poiščite v prosojnicah s predavanj.

Namig: Implementacija je kratka, če  uporabite funkciji `enc_shift(k, pt)` ter `dec_shift(k, ct)`.


In [ ]:
def enc_vigenere(key, pt):
    pass

In [ ]:
assert enc_vigenere("tajno", "dober dan") == "žokšfšdjc"
assert enc_vigenere("b", "dober dan") == enc_shift("b", "dober dan")

In [ ]:
def dec_vigenere(key, ct):
    pass

In [ ]:
for k, pt in [("fri", "pozdravljen svet"), ("sladoled", "kratke hlače in natikači"), ("fleifnjvlauierhfdlejkfjsčawšžćčdqšfl", "kratek tajnopis")]:
    assert dec_vigenere(k, enc_vigenere(k, pt)) == pt

## Naloga 3: Implementacija zamenjalne šifre

Implementirajte algoritma šifriranja in dešifriranja za zamenjalno šifro. Pri zamenljalni šifri je ključ permutacija, ki pove, kako se znaki čistopisa zamenjajo z znaki tajnopisa. Kot primer je podan ključ $k$

$$k=\left(\begin{array}{}
a & b & c & \dots & ž & \_  \\
i & r & o & \dots & g & z \\
\end{array}\right)
$$

Ključ k zamenja črko `a` s črko `i`, črko `b` s črko `r` itd. Naravni način implementacije take permutacije s slovarjem. Funkcija `gen_substitution_key()` je primer funkcije, ki ustvari naključno permutacijo.

In [ ]:
import random

def gen_substitution_key():
    return {p:c for p, c in zip(ALPHABET, random.sample(ALPHABET, len(ALPHABET)))}
    # ali še krajše:  dict(zip(ALPHABET, random.sample(ALPHABET, len(ALPHABET))))

In [ ]:
def enc_substitution(key, pt):
    pass

In [ ]:
# Nekaj testov zamikalne šifre
k_subst = {
    'a': 'n', 'b': 'j', 'c': 'g', 'č': 'l', 'd': 'ž', 'e': 'c', 'f': 'u', 'g': 'k', 'h': 'p',
    'i': 'f', 'j': 'z', 'k': 'i', 'l': 'e', 'm': 't', 'n': 'š', 'o': 'm', 'p': ' ', 'r': 'b',
    's': 'r', 'š': 'v', 't': 'a', 'u': 'č', 'v': 'o', 'z': 's', 'ž': 'h', ' ': 'd'
}

assert enc_substitution(k_subst, "dober dan") == "žmjcbdžnš"
assert enc_substitution(k_subst, "lahko noč") == "enpimdšml"

In [ ]:
def dec_substitution(key, ct):
    pass

In [ ]:
# Nekaj testov zamikalne šifre
for pt in ["dober dan", "dobro jutro", "lahko noč"]:
    k = gen_substitution_key()
    assert dec_substitution(k, enc_substitution(k, pt)) == pt

## Kriptoanaliza in napadi na zgodovinske šifre

Pri napadih na šifre bomo predpostavljali, da je šifrirana vsebina slovenska proza. Zato bomo najprej izračunali preprosto statistiko za tovrstna besedila. To bomo storili tako, da bomo (sicer zelo preprosto) statistično analizirali nekaj slovenskih besedil. Kot primer so izbrana štiri besedila iz slovenske zakonodaje: Obligacijski zakonik, Pomorski zakonik, Zakon o kazenskem postopku ter Stvarnopravni zakonik. Besedila so bila posneta s strani [Uradnega lista,](https://www.uradni-list.si) njihova vsebina pa se nahaja v podimeniku `data` v besedilnih datotekah s končnico `.txt`.

Preberemo njihovo vsebino, s pomočjo funkcije `normalize` odstranimo vse znake, ki niso del naše abecede, in tako očiščeno besedilo shranimo.

In [ ]:
def load(filename):
    with open(filename, encoding="utf-8") as h:
        return h.read()
    
slovenian_corpus = normalize(load("data/oz.txt") + " " + load("data/zs.txt") + " " + \
                             load("data/pz.txt")+ " " + load("data/zkp.txt"))

### Frekvenčna analiza

Zanima nas predvsem pogostost pojavitve posameznih znakov v naši abecedi. To dosežemo z uporabo razreda `collections.Counter`

In [ ]:
from collections import Counter

# prešteje pojavitve znakov v podanem niz
freqs_slovene = Counter(slovenian_corpus)
print("Frekvence (absolutne) znakov v Slovenščini")
print(freqs_slovene)

# Absolutne frekvence pretvorimo v relativne
total = sum(freqs_slovene.values())
relative_freqs_slovene = {k:v/total for k, v in freqs_slovene.items()}

print("Relativne frekvence znakov v Slovenščini:")
print(relative_freqs_slovene)

Porazdelitev znako lahko grafično predstavimo s pomočjo knjižnice `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

def plot_freqs():
    stats = dict(sorted(relative_freqs_slovene.items())) # Slovar sortiramo po abecednem vrstnem redu, da bo prikaz lepši

    x = list(stats.keys())
    y = list(stats.values())

    plt.bar(x, y)

    plt.xlabel('Znak')
    plt.ylabel('Odstotek')
    plt.title('Porazdelitev znakov v slovenskih besedilih')
    plt.grid()
    
plot_freqs()

## Naloga 4: Napad s surovo silo na zamično šifro

Pri tej nalogi bomo izrabili dejstvo, da ima zamična šifra trivialno majhen prostor ključev. Zašifrirano vsebino bomo dešifrirali z vsemi možnimi ključi in kot pravi ključ izbrali tistega, pri katerem je dešifrirana vsebina *smiselna*.

V datoteki `data/ct-shift.txt` se nahaja tajnopis šifriran z zamično šifro in neznanim ključem. S tem napadom bomo ugotovili pravi ključ in prebrali vsebino.

Implementirajte funkcijo `bf_shift(ct)`, ki podani tajnopisa dešifrira z vsemi možnimi ključi. Kot rezultat na funkcija vrne seznam terk, kjer prvi element posamezne terke predstavlja šifrirni ključ, drugi element pa nekaj prvih (npr. 50) znakov dešifriranega tajnopisa. Uporabnik se lahko nato na podlagi izpisa odloči, kateri ključ je pravi.

In [ ]:
def bf_shift(ct):
    pass

bf_shift(
    load("data/ct-shift.txt") # preberemo tajnopis iz datoteke   
) # dešifriramo z vsemi mogočimi ključi

## Naloga 5: Avtomatiziran napad s surovo silo na zamično šifro

Sedaj bomo pristop iz naloge 4 izboljšali, in sicer tako, da bo algoritem ne bo več
potreboval pomoči uporabnika, temveč bo samodejno ugotovil, kateri ključ je pravi.

To bomo naredili tako, da bomo primerjali porazdelitev znakov v običajni Slovenščini s porazdelitvijo 
znakov v dešifriranemu besedilu. Tisto dešifrirano besedilo, katerega porazdelitev znakov je najbolj
podobna porazdelitvi znakov v običajni Slovenščini, je z veliko verjetnostjo bilo dešifrirano s pravim
ključem.

### Naloga 5.1: Implementacija funkcije `chi_sqr(freqs, rel_freqs)`

Naloga bo lažje rešljiva s pomočjo pomožne funkcije `chi_sqr(freqs, rel_freqs)`, ki vzame dva slovarja.
Prvi, `freqs`, ponazarja **absolutne** frekvence črk v dešifriranem besedilu, drugi, `rel_freqs`, pa
**relativne** frekvence črk v ciljnem jeziku, npr. Slovenščini. Funkcija na izhodu vrne vrednost
statistike $\chi^2$.

Podrobnosti izračuna omenjene statistike poiščite v prosojnicah s predavanj. Primer:
```python
>>> f = {"a": 340, "b": 620, "c": 100}
>>> p = {"a": 0.3, "b": 0.5, "c": 0.2}
>>> chi_sqr(f, p)
75.9748427672956
```

Predpostavite, da podana slovarja vsebujeta iste ključe.

In [ ]:
def chi_sqr(f, p):
    pass

In [ ]:
assert abs(chi_sqr({"a": 340, "b": 620, "c": 100}, {"a": 0.3, "b": 0.5, "c": 0.2}) - 75.9748427672956) < 0.00001

### Naloga 5.2: Avtomatizirano iskanje pravega ključa

Implementirajte funkcijo `bf_shift_auto(ct, rel_freq)`, ki iz tajnopisa ter relativnih frekvenc ciljnega jezika
ugotovi najbolj verjeten ključ, ki je bil uporabljen za šifriranje tajnopisa.

Pri izdelavi statistike nad dešifriranim besedilom si pomagajte z razredom `Counter`.

Funkcija naj vrne par `(k, pt)`, kjer `k` predstavlja ključ, `pt` pa prvih 50 znakov dešifriranega besedila.

In [ ]:
def bf_shift_auto(ct, rel_freqs):
    pass

bf_shift_auto(load("data/ct-shift.txt"), relative_freqs_slovene)

## Naloga 6: Napad na Vigenerjevo šifro

Na koncu implementirajmo še celoten napad na Vigenerjevo šifro. Implementacija bo potekala v dveh delih.

### Naloga 6.1: Napad, ko je dolžina ključa znana

Denimo, da je dolžina ključa $q$. Torej vemo, da se vsaki $q$-ti znak šifrira z isto zamično šifro. 

Pri napadu moramo tajnopis razdeliti v $q$ pod-tajnopisov, nato pa vsakega od teh razbiti z avtomatiziranim napadom na zamično šifro. Ko je znan ključ za vsakega od pod-tajnopisov, ključe združimo v celoto in cel tajnopis dešifriramo s Vigenerjevim dešifriranim algoritmom in sestavljenim ključem. Podrobnosti poiščite na prosojnicah s predavanj.

Funkcija `bf_vigenere(klen, ct, rel_freqs)` vzame kot prvi argument `klen` dolžino ključa, `ct` predstavlja tajnopis, `rel_freqs` pa relativne frekvence znakov v ciljnem jeziku (v Slovenščini v našem primeru).

Tajnopis je shranjen v datoteki `data/ct-vigenere.txt`, dolžina ključa pa je 26.

In [ ]:
def bf_vigenere(klen, ct, rel_freqs):
    pass

# dešifriraj tajnopis
bf_vigenere(26, load("data/ct-vigenere.txt"), relative_freqs_slovene)

### Naloga 6.2: Napad, ko dolžina ključa ni znana

Na koncu ostane še primer, ko napadalec dolžine ključa ne pozna. V tem primeru mora napadalec le preskusiti vse ključe od dolžine 1 pa do neke zgornje meje $q_\max$, katero si zastavi sam.

Ko najde ključ, ki tajnopis dešifrira v neko smisleno besedilo, je naloga končana. Pri ugotavljanju, ali je vsebina smiselna, uporabite statistiko $\chi^2$ oz. funkcijo `chi_sqr(f, p)`.

Funkcija `break_vigenere(ct, rel_freqs, max_klen=30)` na vhodu vzame tajnopis `ct`, relativne frekvence znakov ciljnega jezika `rel_freqs` ter opcijski argument `max_klen`, ki podaja zgornjo mejo dolžine ključa t. i. $q_\max$. Funkcija naj vrne par `(k, pt)`, ki predstavlja ključ in dešifriran tajnopis.

Primer tajnopisa šifriranega z neznano dolžino ključa se nahaja v datoteki `data/ct-vigenere-2.txt`.

In [ ]:
def break_vigenere(ct, rel_freqs, max_klen=30):
    pass

break_vigenere(load("data/ct-vigenere-2.txt"), relative_freqs_slovene)